In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import pandas as pd
import itertools
import os
import ROOT
import datetime
from root_numpy import root2array, root2rec, tree2rec, array2root
from scipy.optimize import curve_fit
from scipy.misc import factorial

In [ ]:
plt.rcParams.update({"font.size": 18})
# matplotlib.rcParams['backend'] = 'cairo'

In [ ]:
from ELOSS import dpdx as DPDX
from ELOSS import dedx as DEDX

In [ ]:
from scipy.interpolate import spline

pdg_rr_v = []
pdg_dedx_v = []
pdg_ke_v = []
pdg_rr_fit_v = []
pdg_dedx_fit_v = []
pdg_ke_fit_v = []

# fin = open('/home/david/Neutrinos/StopMuCalibration/mutable.txt')
fin = open('./data/mutable.txt')
rho = 1.396

for line in fin:
    words = line.split()
    pdg_rr_v.append(   float(words[-1])/rho )
    pdg_dedx_v.append( float(words[2]) *rho )
    pdg_ke_v.append(   float(words[0])      )


pdg_rr_fit_v   = np.linspace(1,1000,1000)
pdg_dedx_fit_v = spline(pdg_rr_v,pdg_dedx_v,pdg_rr_fit_v)
pdg_ke_fit_v = spline(pdg_rr_v,pdg_ke_v,pdg_rr_fit_v)


# fiven a RR value, get the energy
def ERange(RR):
    for i in xrange(len(pdg_rr_fit_v)):
        if (RR <= pdg_rr_fit_v[i]):
            return pdg_ke_fit_v[i]              
    return -1

print ERange(100.)

In [ ]:
#df = pd.read_pickle('stopmu.pkl')
#df = pd.read_pickle('stopmu_data_mcc83.pkl') # data
data = pd.read_csv('stopmu_mcc9_data.csv',sep='\t')
print data.shape

In [ ]:
data.columns = ["rr", "dqdx", "pitch", "px", "py", "pz"]

In [ ]:
fig = plt.figure(figsize=(12, 6))
BINS = (np.linspace(0, 250, 100), np.linspace(150, 500, 100))
plt.hist2d(data["rr"].values, data["dqdx"].values, bins=BINS)
plt.grid()
plt.xlabel("Residual Range [cm]", fontsize=20, fontweight="bold")
plt.ylabel("dQ/dx [ADC / cm]", fontsize=20, fontweight="bold")
# plt.title('Tagged Stopping Muon Profile [MC Cosmics]',fontsize=20,fontweight='bold')
plt.title(
    "Tagged Stopping Muon Profile [BNB EXT Run 4a]", fontsize=20, fontweight="bold"
)
plt.savefig("./figs/tagged/stopping_mu_rr_profile.pdf", bbox_inches="tight")
plt.savefig("./figs/tagged/stopping_mu_rr_profile.png", bbox_inches="tight")
plt.show()

In [ ]:
# invert Recombination Modified Box Model to get dE/dx from dQ/dx

# argon density [g/cm^3]
rho = 1.396
# electric field [kV/cm]
efield = 0.273
# ionization energy [MeV/e]
Wion = 23.6 * (10 ** (-6))

fModBoxA = 0.93
fModBoxB = 0.562


def ModBoxInverse(dqdx):
    dedx = (np.exp(fModBoxB * Wion * dqdx) - fModBoxA) / fModBoxB
    return dedx

In [ ]:
def dEdx(x, elecgain):
    dqdx = x["dqdx"] * elecgain
    return ModBoxInverse(dqdx)

In [ ]:
# fin = open('/home/david/Neutrinos/StopMuCalibration/mutable.txt')
fin = open("./data/mutable.txt")
rho = 1.396
pdg_rr_v = []
pdg_dedx_v = []
for line in fin:
    words = line.split()
    pdg_rr_v.append(float(words[-1]) / rho)
    pdg_dedx_v.append(float(words[2]) * rho)

In [ ]:
from scipy.interpolate import spline

pdg_rr_fit_v = np.linspace(1, 1000, 1000)
pdg_dedx_fit_v = spline(pdg_rr_v, pdg_dedx_v, pdg_rr_fit_v)

In [ ]:
# pylandau from https://github.com/SiLab-Bonn/pylandau
from pylandau import langau
from scipy.optimize import curve_fit

guess = [4.0, 0.2, 0.2]  # MPV, Landau sigma, gauss sigma in that order


def GL(x_v, mpv, sL, sG, A):
    return A * langau(x_v * 100.0, mpv * 100, sL * 100, sG * 100)

Compute and minimise $\chi^2$ as a function of electronics gain. Described as:
$$
        \chi^2 = \frac{1}{N} \frac{\mbox{(MPV theory - MPV measured (gain))}^2}{\sigma^2}
$$

In [ ]:
# calculate chi^2 of PDG - data as a function of electronics gain applied

elec_gain = np.linspace(230,250,20)
chisq_v  = []

rr_ranges = np.linspace(100,150,10)

BINS = np.linspace(1,6.0,100)

datafit = data.query('rr > 100 and rr < 150 and pitch > 0.3 and pitch < 0.4 and px < 0.2 and px >-0.2')
print datafit.shape

for elecgain in elec_gain:
    
    datafit['dedx'] = datafit.apply(lambda x : dEdx(x,elecgain),axis=1)
    
    chilocal = 0.
    
    for n in xrange(len(rr_ranges)-1):
        
        rrmin = rr_ranges[n]
        rrmax = rr_ranges[n+1]
        
        rravg = 0.5*(rrmax+rrmin)
        
        dftmp = datafit.query('rr > %i and rr < %i'%(rrmin,rrmax))
        
        dedx_v = dftmp['dedx'].values
    
        vals,bine = np.histogram(dedx_v,bins=BINS)
        binc = 0.5*(bine[1:]+bine[:-1])
        guess = [1.8,0.1,0.1,30.]
        popt,popv = curve_fit(GL,binc,vals,p0=guess,bounds=([1.2,0.02,0.02,1],[4.0,0.3,0.3,200]))#,sigma=np.sqrt(vals),absolute_sigma=True)
        
        mpv_data = popt[0]
        mpv_err  = np.sqrt(np.diag(popv))[0]
        
        #print 'norm : %f and N entries : %i'%(popt[-1],dftmp.shape[0])
        
        mpv_theory = DPDX(ERange(rravg),0.35,105.6)
        
        #print '\t @ gain %i w/ RR %.0f ->  MPV measured : %.02f \t theory : %.02f \t err : %.02f'\
        #%(elecgain,rravg,mpv_data,mpv_theory,mpv_err)
        
        chilocal += ( (mpv_data-mpv_theory)**2 ) / (mpv_err**2 + 0.015**2 + 0.01**2)
        
    print 'Gain = %.02f -> chisq = %.02f'%(elecgain,chilocal)
    
    chisq_v.append(chilocal/float(len(rr_ranges)-1.))

In [ ]:
chi_min = 100000.
gain_min = 0.
for i,chi in enumerate(chisq_v):
    
    if (chi < chi_min):
        
        chi_min  = chi
        gain_min = elec_gain[i]


fig = plt.figure(figsize=(10,6))

min1sigma = 1000
max1sigma = 0
for i,g in enumerate(elec_gain):
    chi = chisq_v[i]
    chidiff = chi - chi_min
    if (chidiff < 1):
        if (g > max1sigma): max1sigma = g
        if (g < min1sigma): min1sigma = g

plt.plot(elec_gain,chisq_v,'bo--')
plt.xlabel('Electronics Gain [$e^-$/ADC]',fontsize=20,fontweight='bold')
plt.ylabel('$\chi^2$ / d.o.f.',fontsize=20,fontweight='bold')
plt.axvline(gain_min,color='k',lw=3,label='$\chi^2$ = %.02f @ gain = %.01f $e^-$/ADC'%(chi_min,gain_min))
plt.title("[MicroBooNE Preliminary Run 4a EXT]", loc="right", fontsize=14)
plt.grid()
# plt.title("MicroBooNE Preliminary Run4a Data", loc="right", color="darkgrey")
# plt.xlim([245,257])
# plt.ylim([0,30])
plt.ylim([0,5])
plt.axhspan(chi_min,chi_min+1,color='k',alpha=0.2,label='+1 $\sigma$ interval : [%.01f,%.01f]'%(min1sigma,max1sigma))
plt.legend(loc=1,fontsize=18)
plt.savefig("./figs/tagged/chi2_v_egain_clipped.png",bbox_inches='tight')
plt.savefig("./figs/tagged/chi2_v_egain_clipped.pdf",bbox_inches='tight')
plt.show()
print gain_min

In [ ]:
data["dedx"] = data.apply(lambda x: dEdx(x, gain_min), axis=1)

In [ ]:
print(data["rr"].min(), data["rr"].max())

In [ ]:
fig = plt.figure(figsize=(10, 6))
BINS = (np.linspace(0, 150, 100), np.linspace(1, 4, 100))
plt.hist2d(data["rr"].values, data["dedx"].values, bins=BINS)
plt.grid()
plt.xlabel("Residual Range [cm]", fontsize=20, fontweight="bold")
plt.ylabel("dE/dx [MeV / cm]", fontsize=20, fontweight="bold")
plt.title("Tagged Stopping Muon Profile [DATA]", fontsize=20, fontweight="bold")
plt.savefig("./figs/tagged/rr_v_dqdx.png", bbox_inches="tight")
plt.show()

In [ ]:
### rr_range_v = np.linspace(10,150,40)

BINS = np.linspace(1,6.0,50)
xvals = np.linspace(1,6,1000)

mpv_v = []
mpv_e = []
rr_v = []
rr_ranges = np.linspace(10,200,40)

for n in xrange(len(rr_ranges)-1):
    
    rrmin = rr_ranges[n]
    rrmax = rr_ranges[n+1]
    
    dftmp = data.query('rr > %i and rr < %i and pitch < 0.4 and pitch > 0.3 and px < 0.2 and px >-0.2'%(rrmin,rrmax))
    
    dedx_v = dftmp['dedx'].values
    
    vals,bine = np.histogram(dedx_v,bins=BINS)
    binc = 0.5*(bine[1:]+bine[:-1])
    guess = [1.8,0.1,0.1,30.]
    popt,popv = curve_fit(GL,binc,vals,p0=guess,bounds=([1.2,0.02,0.02,1],[4.0,0.3,0.3,1000]))#,sigma=np.sqrt(vals),absolute_sigma=True)
    print popt
    
    pope = np.sqrt(np.diag(popv))
    
    if (pope[0]/popt[0] > 0.1):
        continue
    
    mpv_v.append(popt[0])
    mpv_e.append(pope[0])
    rr_v.append(0.5*(rrmin+rrmax))
    
    fig = plt.figure(figsize=(6,6))
    plt.xlabel('dE/dx [MeV/cm]',fontsize=20,fontweight='bold')
    plt.errorbar(binc,vals,yerr=np.sqrt(vals),fmt='bo',lw=2)
    plt.title('Residual Range [%i,%i]'%(rrmin,rrmax),fontsize=20,fontweight='bold')
    plt.plot(xvals,GL(xvals,*popt),'r--',lw=2,label='MPV = %.02f MeV/cm'%(popt[0]))
    plt.grid()
    plt.legend(loc=1)
    plt.savefig('./figs/tagged/rr_slices/rr_slice_%i-%i.pdf'%(rrmin,rrmax), bbox_inches='tight')
    plt.savefig('./figs/tagged/rr_slices/rr_slice_%i-%i.png'%(rrmin,rrmax), bbox_inches='tight')
    plt.show()

In [ ]:
fig = plt.figure(figsize=(6, 6))
plt.errorbar(rr_v, mpv_v, yerr=mpv_e, fmt="bo", lw=2, markersize=8)
plt.grid()
plt.xlabel("RR [cm]")
plt.ylabel("Fitted MPV [MeV/cm]")
plt.show()

In [ ]:
rr_v_truth = np.linspace(10, 200, 100)
mpv_v_truth_03 = []
mpv_v_truth_035 = []
mpv_v_truth_04 = []
for rr in rr_v_truth:
    mpv_theory_03 = DPDX(ERange(rr), 0.3, 105.6)
    mpv_theory_035 = DPDX(ERange(rr), 0.35, 105.6)
    mpv_theory_04 = DPDX(ERange(rr), 0.4, 105.6)
    mpv_v_truth_03.append(mpv_theory_03)
    mpv_v_truth_035.append(mpv_theory_035)
    mpv_v_truth_04.append(mpv_theory_04)

In [ ]:
len(mpv_v)

In [ ]:
fig = plt.figure(figsize=(10, 6))
plt.errorbar(
    rr_v, mpv_v, yerr=mpv_e, fmt="bo", lw=2, markersize=8, label="fitted tracks"
)
plt.plot(rr_v_truth, mpv_v_truth_03, "k--")
plt.plot(rr_v_truth, mpv_v_truth_04, "k--", label="theory")
plt.axvspan(100, 150, color="k", alpha=0.1, label="fit range")
plt.grid()
plt.legend(loc=1)
# plt.title("MCC9 data", loc="left")
plt.title("[MicroBooNE Preliminary Run 4a EXT]", loc="right", fontsize=14)
plt.xlabel("RR [cm]")
plt.ylabel("Fitted MPV [MeV/cm]")
plt.savefig("./figs/tagged/rr_v_fitted_mpv.png", bbox_inches="tight")
plt.savefig("./figs/tagged/rr_v_fitted_mpv.pdf", bbox_inches="tight")

plt.show()

In [ ]:
# mpv_v_ratio = [mpv_v[i] / mpv_v_truth_035[i] for i in range(len(mpv_v_truth_035))]

fig, axs = plt.subplots(2, 1, sharex=True, figsize=(10,8), gridspec_kw={'height_ratios':[0.7,0.3]})
# Remove horizontal space between axes
fig.subplots_adjust(hspace=0)

# Plot each graph, and manually set the y tick values
axs[0].errorbar(
    rr_v, mpv_v, yerr=mpv_e, fmt="bo", lw=2, markersize=8, label="fitted tracks"
)
axs[0].plot(rr_v_truth, mpv_v_truth_03, "k--")
axs[0].plot(rr_v_truth, mpv_v_truth_04, "k--", label="theory")
axs[0].axvspan(100, 150, color="k", alpha=0.1, label="fit range")
axs[0].set_ylabel("Fitted MPV [MeV/cm]")
axs[0].grid()
# axs[0].set_yticks(np.arange(-0.9, 1.0, 0.4))
# axs[0].set_ylim(-1, 1)

axs[0].set_title("[MicroBooNE Preliminary Run 4a EXT]", loc="right", fontsize=14)
axs[0].plot(rr_v_truth, mpv_v_truth_03, "k--")
axs[0].plot(rr_v_truth, mpv_v_truth_04, "k--")
axs[1].axvspan(100, 150, color="k", alpha=0.1, label="fit range")
# axs[1].set_yticks(np.arange(0.1, 1.0, 0.2))
axs[1].set_ylim(-5, 5)
axs[1].set_xlim(0, 200)
axs[1].set_ylabel("% diff")
axs[1].set_xlabel("Residual Range [cm]")
axs[1].grid()

plt.show()

plt.show()

In [ ]:
fout = open("/home/anchovy/uni/gain-calibration/dedx_vs_rr_data.txt", "w+")
for i, rr in enumerate(rr_v):
    mpv = mpv_v[i]
    mpve = mpv_e[i]
    fout.write("%.04f %.04f %.04f \n" % (rr, mpv, mpve))
fout.close()

In [ ]:
finfit = open("/home/anchovy/uni/gain-calibration/dedx_vs_rr_mcc9.txt", "r")
rr_d = []
mpv_v_d = []
mpv_e_d = []
for line in finfit:
    words = line.split()
    rr_d.append(float(words[0]))
    mpv_v_d.append(float(words[1]))
    mpv_e_d.append(float(words[2]))

In [ ]:
fig = plt.figure(figsize=(10, 6))
plt.errorbar(rr_v, mpv_v, yerr=mpv_e, fmt="bo", lw=2, markersize=8, label="data")
plt.errorbar(
    rr_d, mpv_v_d, yerr=mpv_e_d, fmt="ro", lw=2, markersize=8, label="monte carlo"
)
plt.plot(rr_v_truth, mpv_v_truth_03, "k--", lw=2)
plt.plot(rr_v_truth, mpv_v_truth_04, "k--", lw=2, label="theory")
plt.axvspan(100, 150, color="k", alpha=0.1, label="fit range")
plt.grid()
plt.legend(loc=1)
plt.title("MCC9 Stopping Muon Calibration")
plt.xlabel("RR [cm]")
plt.ylabel("Fitted MPV [MeV/cm]")
plt.savefig("./figs/tagged/rr_v_fitted_mpv_wtruth.png", bbox_inches="tight")

plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.hist(data['pitch'],bins=np.linspace(0.29,1.0,30),histtype='step',lw=2)
plt.grid()
plt.xlabel('Pitch [cm]',fontsize=20,fontweight='bold')
plt.ylabel('Entries',fontsize=20,fontweight='bold')
plt.title('Pitch Distribution',fontsize=20,fontweight='bold')
plt.axvline(0.4,lw=4,color='k',linestyle='--')
plt.text(0.42,22000,r'$\leftarrow$ cut',fontsize=26,fontweight='bold')
plt.savefig("./figs/tagged/pitch_dist.png",bbox_inches='tight')

plt.show()

print np.median(data['pitch'].values)
print np.average(data['pitch'].values)

In [ ]:
#rr_range_v = np.linspace(10,150,40)

BINS = np.linspace(1,6.0,100)
xvals = np.linspace(1,6,1000)

pitch_ranges_v = [[0.3,0.33,0.36,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.9],\
                 [0.3,0.4,0.5,0.6,0.7,0.8,0.9],\
                 [0.4,0.5,0.6,0.7,0.8,0.9],\
                 [0.4,0.5,0.6,0.7,0.8,0.9],\
                 [0.5,0.6,0.7,0.8,0.9]]

#pitch_ranges_v = [[0.3,0.35,0.4,0.5,0.6,0.7,0.8,0.9],\
#                [0.3,0.35,0.4,0.5,0.6,0.7,0.8,0.9],\
#                [0.4,0.5,0.6,0.7,0.9]]

px_ranges = [0,0.2,0.4,0.6,0.8,1.0]#0.2,0.4,0.8]

fout = open('/home/anchovy/uni/gain-calibration/dedx_vs_pitch_data.csv','w+')

for m in xrange(len(px_ranges)-1):
    
    pxmin = px_ranges[m]
    pxmax = px_ranges[m+1]
    
    dfpx = data.query('rr > 100 and rr < 150. and ((px > %f and px < %f) or (px > %f and px < %f))'%\
    (pxmin,pxmax,-pxmax,-pxmin))
    
    mpv_v = []
    mpv_e = []
    pitchh_v = []
    pitchl_v = []
    pitch_v = []

    for n in xrange(len(pitch_ranges_v[m])-1):
    
        pmin = pitch_ranges_v[m][n]
        pmax = pitch_ranges_v[m][n+1]
    
        pitchh_v.append(pmax)
        pitchl_v.append(pmin)
        pitch_v.append(0.5*(pmin+pmax))
    
        dftmp = dfpx.query('rr > 100 and rr < 150 and pitch > %f and pitch < %f'%(pmin,pmax))
    
        dedx_v = dftmp['dedx'].values
    
        vals,bine = np.histogram(dedx_v,bins=BINS)
        binc = 0.5*(bine[1:]+bine[:-1])
        guess = [1.7,0.1,0.1,5000.]
        try:
            popt,popv = curve_fit(GL,binc,vals,p0=guess)#,sigma=np.sqrt(vals),absolute_sigma=True)
            print popt
        except:
            popt=guess
    
        pope = np.sqrt(np.diag(popv))
    
        mpv_v.append(popt[0])
        mpv_e.append(pope[0])
    
        fig = plt.figure(figsize=(10,6))
        plt.errorbar(binc,vals,yerr=np.sqrt(vals),fmt='bo',lw=2)
        plt.title('Pitch range [%.02f,%.02f]'%(pmin,pmax))
        plt.plot(xvals,GL(xvals,*popt),'r--',lw=2)
        plt.grid()
        plt.show()
        
        
        fout.write('%.04f\t%.04f\t%.04f\t%.04f\t%.04f\t%.04f\t%.04f\n'%\
                   (pxmin,pxmax,0.5*(pmin+pmax),pmin,pmax,popt[0],pope[0]))
        
    fig = plt.figure(figsize=(6,6))
    plt.errorbar(pitch_v,mpv_v,yerr=mpv_e,fmt='bo',lw=2,markersize=8)
    plt.grid()
    plt.xlabel('Pitch [cm]')
    plt.ylabel('Fitted MPV [MeV/cm]')
    plt.title('Px in range [%.02f,%.02f]'%(pxmin,pxmax))
    plt.show()

fout.close()